In [ ]:
from track_utils_3_0_2 import *

In [ ]:
with h5py.File('D:/Tracking/Videos/20250530_UMN026/3/20250530_UMN026_3_2.h5', 'r') as hf:
    video = hf['data'][:]

In [62]:
view(video)

In [ ]:
imgs, saturations, MCV, x, hbmass= main_run(video)

In [ ]:
sat=np.stack( saturations, axis=0 )
MCVs=np.stack( MCV, axis=0 )
MCHs=np.stack( hbmass, axis=0 )
sat[(sat > 1) | (sat < -0.2)] = np.nan
MCHC=(MCHs/MCV)*(10**14)
MCVs[(MCVs > 150) | (MCVs < 30)] = np.nan
MCHs[(MCHs > 5e-11) | (MCHs < -0)] = np.nan
MCHs=MCHs*1e12
d = {'sat': sat, 'MCV': MCVs, 'MCH': MCHs, 'MCHC': MCHC}
sat_df = pd.DataFrame(data=d)
sat_df=sat_df.dropna()
sat_df['sat_mean']=sat_df['sat'].rolling(window=20).mean() 
sat_df['MCV_mean']=sat_df['MCV'].rolling(window=20).mean() 
sat_df['MCHC_mean']=sat_df['MCHC'].rolling(window=20).mean() 
sat_df['MCH_mean']=sat_df['MCH'].rolling(window=20).mean() 
fig, ax = plt.subplots()
# ax.set_ylim(0, 1)
plt.plot(sat_df['sat'])

In [ ]:
model=keras.models.load_model('CNN/model_ResNet50_A01.h5', safe_mode=False) #load the model
preds=CNN(imgs,model)